In [1]:

import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
from numpy import *
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
import math
from collections import Counter

In [5]:
import time

reward_list= []
GLOBAL_UAV1_PATH = []
GLOBAL_UAV2_PATH = []
pre_total_reward = 1000
class uavEnv(Env):
    def __init__(self):
        self.X_lau_location  = [950, 50]
        self.Y_lau_location  = [950, 50]
        self.UAV1_PATH = []
        self.UAV2_PATH = []
        self.UAV3_PATH = []
        self.UAV4_PATH = []
        self.start_time = time.time()
        self.g31_capacity = {}
        self.completed = 0
        self.account = 0 
        self.E = 0
        self.action_space = MultiDiscrete([5,5]) 
        self.observation_space = Box(low=np.array([-100]*4),high=np.array([1000]*4)) 
        self.state = np.array([10]*4)
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        # number of users
        self.User_Location_X = [399, 91, 750, 956, 393, 790, 728, 211, 565, 971, 576, 266, 571, 401, 875, 837, 575, 585, 625, 788, 810, 792, 499, 140, 816, 391, 29, 36, 171, 530, 58, 546, 205, 89, 964, 117, 818, 64, 680, 981]
        self.User_Location_Y = [29, 590, 9, 448, 480, 633, 862, 588, 860, 103, 680, 86, 911, 948, 198, 933, 824, 270, 903, 825, 380, 641, 444, 725, 630, 150, 852, 382, 362, 463, 722, 621, 971, 420, 863, 853, 807, 317, 699, 991]

        Number_User = 40
        self.Max_flow_total = 0
        self.total_reward = 0

        
    def step(self, Action):

        M_LAU_length = 50
        Number_LAU = 2
        pently_lau = 0
        
    

        for i in range(Number_LAU):
            if Action[i] == 0:  #hover
                self.X_lau_location[i] = self.X_lau_location[i]
                self.Y_lau_location[i] = self.Y_lau_location[i]

            elif Action[i] == 1: # back
                self.Y_lau_location[i] = self.Y_lau_location[i] - M_LAU_length
                if self.Y_lau_location[i]<0: # Out of range
                    pently_lau = 1
                    self.Y_lau_location[i] = self.Y_lau_location[i] + M_LAU_length

            elif Action[i] == 2: # forward
                self.Y_lau_location[i] = self.Y_lau_location[i] + M_LAU_length
                if self.Y_lau_location[i]>1000:
                    pently_lau = 1
                    self.Y_lau_location[i] = self.Y_lau_location[i] - M_LAU_length

            elif Action[i] == 3: # left
                self.X_lau_location[i] = self.X_lau_location[i] - M_LAU_length
                if self.X_lau_location[i]<0:
                    pently_lau = 1
                    self.X_lau_location[i] = self.X_lau_location[i] + M_LAU_length

            elif Action[i] == 4: # right
                self.X_lau_location[i] = self.X_lau_location[i]+ M_LAU_length
                if self.X_lau_location[i]>1000:
                    pently_lau = 1
                    self.X_lau_location[i] = self.X_lau_location[i] - M_LAU_length

        Number_User =  len(self.User_Location_X)
        R_User_Location_X = []
        R_User_Location_Y = []
        Current_reward = 0
        for j in range(Number_User):
            for i in range(Number_LAU):
                D_GL = math.sqrt((self.User_Location_X[j]-self.X_lau_location[i])**2+(self.User_Location_Y[j]-self.Y_lau_location[i])**2)/1000
                if D_GL<0.1: # coverage of UAV :100m
                    self.completed = self.completed + 1
                    Current_reward = Current_reward + 10
                    R_User_Location_X.append(self.User_Location_X[j])
                    R_User_Location_Y.append(self.User_Location_Y[j])

            
          
        self.User_Location_X = [x for x in self.User_Location_X if x not in R_User_Location_X]     
        self.User_Location_Y = [x for x in self.User_Location_Y if x not in R_User_Location_Y]   
                    
                    
        
        self.account = self.account + 1
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        U1 =[self.X_lau_location[0],self.Y_lau_location[0]]
        U2 =[self.X_lau_location[1],self.Y_lau_location[1]]
        self.UAV1_PATH.append(U1)
        self.UAV2_PATH.append(U2)

        
        
        reward =  Current_reward-(pently_lau)*100
        ##print(reward,self.account,self.completed)
        self.total_reward = self.total_reward + reward
        if self.account >= 200: # maximum step of UAVs
            done = True
            print(self.completed)
            reward_list.append(self.total_reward)
            info = {}
            return self.state, reward, done, info
        
        else:
            done = False
            
        global pre_total_reward
        global GLOBAL_UAV1_PATH
        global GLOBAL_UAV2_PATH
        if self.completed >= 40:
            done = True
            reward = reward+ 20*(100-self.account)
            self.total_reward = self.total_reward + reward
            if self.account < pre_total_reward :
                pre_total_reward = self.account
                # trajectory of UAVs
                GLOBAL_UAV1_PATH = self.UAV1_PATH
                GLOBAL_UAV2_PATH = self.UAV2_PATH
            reward_list.append(self.total_reward)
            info = {}
            print(self.account,self.total_reward)
            return self.state, reward, done, info
        else:
            done = False
        

        info = {}
        return self.state, reward, done, info

    def render(self):
        pass 
    
    
    def reset(self):
        self.account = 0
        self.completed = 0
        self.total_reward = 0
        self.Max_flow_total = 0
        self.average_delay = 0
        self.X_lau_location = []
        self.Y_lau_location = []
        # reset the initial locations of UAVs
        for _ in range(2):
            self.X_lau_location.append(random.randint(0, 1000))
            self.Y_lau_location.append(random.randint(0, 1000))
            
        self.g31_capacity = {}
        self.UAV1_PATH = []
        self.UAV2_PATH = []
        self.UAV3_PATH = []
        self.UAV4_PATH = []
        Number_User = 40
        self.User_Location_X = [399, 91, 750, 956, 393, 790, 728, 211, 565, 971, 576, 266, 571, 401, 875, 837, 575, 585, 625, 788, 810, 792, 499, 140, 816, 391, 29, 36, 171, 530, 58, 546, 205, 89, 964, 117, 818, 64, 680, 981]
        self.User_Location_Y = [29, 590, 9, 448, 480, 633, 862, 588, 860, 103, 680, 86, 911, 948, 198, 933, 824, 270, 903, 825, 380, 641, 444, 725, 630, 150, 852, 382, 362, 463, 722, 621, 971, 420, 863, 853, 807, 317, 699, 991]
        self.E = 0
        self.E_CC = 0
        self.state[0] = self.X_lau_location[0]  
        self.state[1] = self.Y_lau_location[0]
        self.state[2] = self.X_lau_location[1]
        self.state[3] = self.Y_lau_location[1]     
        return self.state


In [14]:
from stable_baselines3 import PPO
from stable_baselines3 import A2C

net_arch = [dict(pi=[128, 128, 128], vf=[128, 128, 128])] 
env = uavEnv()
model = A2C(
    "MlpPolicy",
    env=env, 
    learning_rate=1e-4,)


model.learn(total_timesteps = 2000000)

c:\Users\steve\anaconda3\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
c:\Users\steve\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


22
23
16
22
26
20
25
25
19
20
7
21
26
19
21
10
20
20
22
28
16
22
22
26
18
17
33
27
18
24
21
24
24
13
17
12
20
25
18
24
27
7
25
23
15
15
25
23
30
27
17
19
22
23
18
19
26
18
23
25
18
10
19
18
23
25
23
14
15
15
21
23
27
24
21
24
25
26
23
24
18
18
29
22
22
22
29
5
29
27
25
16
19
14
23
21
22
19
13
26
23
23
29
25
13
10
20
23
26
22
21
21
11
26
15
21
14
24
23
15
20
20
22
27
29
26
26
23
26
24
20
22
18
21
28
24
22
25
24
30
28
28
22
17
27
31
24
24
16
26
25
21
20
23
27
27
24
27
21
19
29
23
27
25
31
29
25
27
10
30
18
25
30
27
14
13
24
23
23
15
23
17
29
25
15
21
23
23
26
14
25
19
32
30
15
25
23
17
7
19
28
18
13
14
17
13
26
29
17
28
22
24
20
25
14
18
13
21
23
20
25
14
18
10
28
11
14
14
22
23
17
15
18
16
28
21
31
28
18
13
11
17
11
17
25
14
28
24
13
18
15
13
19
14
18
25
25
13
22
25
21
23
21
24
24
21
22
24
13
19
28
16
19
16
25
15
19
21
22
14
17
18
18
22
18
23
21
27
20
14
20
10
12
10
27
28
18
22
24
28
9
11
18
24
17
27
16
13
18
10
14
30
18
13
22
15
21
23
26
19
24
24
17
21
17
16
25
23
18
22
19
14
32
29
25


KeyboardInterrupt: 

In [15]:
print(GLOBAL_UAV1_PATH)
print(GLOBAL_UAV2_PATH)

[[869, 935], [819, 935], [769, 935], [719, 935], [719, 885], [719, 835], [669, 835], [619, 835], [569, 835], [619, 835], [619, 785], [569, 785], [519, 785], [469, 785], [419, 785], [419, 735], [419, 685], [419, 635], [419, 585], [419, 535], [419, 485], [419, 435], [419, 385], [419, 335], [419, 285], [419, 235], [419, 185], [419, 135], [469, 135], [469, 85], [519, 85], [569, 85], [619, 85], [669, 85], [719, 85], [769, 85], [819, 85], [869, 85], [919, 85], [919, 135], [969, 135], [969, 185], [969, 235], [969, 235], [969, 285], [969, 335], [919, 335], [919, 385], [919, 435], [869, 435], [869, 485], [819, 485], [769, 485], [719, 485], [669, 485], [619, 485], [569, 485], [519, 485], [469, 485], [419, 485], [369, 485], [319, 485], [269, 485], [219, 485], [169, 485], [169, 435], [169, 385], [169, 335], [169, 285], [169, 235], [169, 185], [219, 185], [269, 185], [319, 185], [369, 185], [419, 185], [469, 185], [519, 185], [569, 185]]
[[244, 270], [194, 270], [144, 270], [94, 270], [94, 320], [9